# Introduction

Avant de commencer, parcourer le fichier README.rst

# Requête HTTP 

Un requête HTTP est une requête basé sur le protocole TCP, elle fait partie de la couche application de la couche OSI. Elle permet d'accéder aux données mise à disposition sur une adresse IP (ou url résolue par un DNS) et un port. 

Les deux ports les plus utilisé dans le web sont le 80 pour les sites en HTTP et le 443 pour les sites en HTTPS. HTTPS est une variable du protocole HTTP basé sur le protocole TLS.

Il existe de nombreux types de requêtes selon la convention `REST`: 
- GET
- POST
- PUT 
- DELETE
- UPDATE.

Dans notre cas nous allons utiliser la plupart du temps des GET et potentiellement des POST. 
- Le GET permet comme sont nom l'indique de récupérer des informations en fonction de certain paramètres. 
- Le POST nécéssite un envoie de données pour récupérer des données. Le body du post est, la plupart du temps, envoyé sous la forme d'un objet JSON.

Ces requêtes encapsulent un certain nombre de paramètres qui permettent soient d'identifier une provenance et un utilisateur ou de réaliser différentes actions.

In [10]:
import requests

In [11]:
url = "http://www.esiee.fr/"
response = requests.get(url)
response.status_code

200

Il existe deux méthodes pour récupérer le contenu de la page :

- `response.text` qui permet de retourner le texte sous la forme d'une chaine de charactères.
- `response.content` qui permet de récupérer le contenu de la page sous la forme de bytes

In [12]:
type(response.content)

bytes

In [13]:
type(response.text)

str

Pour récupérer les 1000 premiers charactères de la page :

In [14]:
response.text[0:1000]

'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2023 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;école de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande école d&#039;ingénieur dans les domaines des transitions numérique, énergétique et environnementale. Classée dans le groupe A, parmi les meilleures écoles d&#039;ingénieur selon le classement de l&#039;Etudiant. Habilitée par la Commission des Titres d&#039;Ingénieur (CTI). Membre de la Conférence des Grandes Ecoles (CGE). " />\n<meta name="viewport" conte

Pour récupérer les headers HTTP de la réponse :

In [15]:
response.headers

{'Date': 'Tue, 21 Nov 2023 12:44:42 GMT', 'Server': 'Apache', 'Content-Language': 'fr', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-UA-Compatible': 'IE=edge', 'X-Content-Type-Options': 'nosniff', 'Content-Length': '15693', 'Content-Type': 'text/html; charset=utf-8', 'X-Varnish': '181918 1643369', 'Age': '10', 'Via': '1.1 varnish (Varnish/7.1)', 'Accept-Ranges': 'bytes', 'Connection': 'keep-alive'}

On peut modifier les paramêtres de la requête et/ou ses headers. On peut par exemple ajouter un UserAgent et un timeout de 10 secondes:

In [16]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers, timeout = 10)
response.content[0:1000]

b'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2023 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;\xc3\xa9cole de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande \xc3\xa9cole d&#039;ing\xc3\xa9nieur dans les domaines des transitions num\xc3\xa9rique, \xc3\xa9nerg\xc3\xa9tique et environnementale. Class\xc3\xa9e dans le groupe A, parmi les meilleures \xc3\xa9coles d&#039;ing\xc3\xa9nieur selon le classement de l&#039;Etudiant. Habilit\xc3\xa9e par la Commission des Titres d&#039;Ing\xc3\xa9nieur (CTI). Membr

## Exercice

In [17]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
class CustomRequest:
    def __init__(self, user_agent="MyUserAgent/5.0", timeout=5, max_retries=3):
        self.user_agent = user_agent
        self.timeout = timeout
        self.max_retries = max_retries
        self.session = self._create_session()

    def _create_session(self):
        session = requests.Session()
        retries = Retry(total=self.max_retries,
                        backoff_factor=0.1,
                        status_forcelist=[500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retries)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        return session

    def _make_request(self, url):
        headers = {'User-Agent': self.user_agent}
        try:
            response = self.session.get(url, headers=headers, timeout=self.timeout)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"Error making request: {e}")
            return None

In [42]:
custom_request = CustomRequest()
response = custom_request._make_request(url)
soup = BeautifulSoup(response.text)
soup.text[0:1000]

"\n\n\n\nESIEE Paris, l'école de l'innovation technologique | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller au contenu\nAller au menu\nPlan du site\n\n\n\n\n\n\n\nBien choisir son école, c'est aussi la rencontrer  ! Rendez-vous à la prochaine journée portes ouvertes le 9 décembre.\n\n\n\n\n\nMasquer l'alerte\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrochuresEspace élèves\n\n\n\nFr\n\n\n\n\nEn\n\n\n\n\n\n\n\n\n\nAfficherMasquer la recherche\r\n\t\t\n\n\n\nSaisissez votre recherche\xa0:\n\nLancer la recherche\n\n\n\nCandidater\n\nAfficherMasquer le menu\n\n\n\n\n\nRetour au menu principalAfficherMasquer le sous menu\xa0: L'écolePourquoi choisir ESIEE Paris ?AfficherMasquer le sous menu\xa0: Gouvernance et conseilsGouvernance et conseilsConseil scientifiqueAfficherMasquer le sous menu\xa0: Départements d'enseignements et de rechercheInformatique et télécommunicationsIngénierie des systèmes cyberphysiquesIngénierie industrielleSanté, énergie et environnement durableManagement, sci

## Exercice 1

- Créer une classe Python permettant de faire des requêtes HTTP.
- Cette classe doit utiliser toujours le même UserAgent.
- Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
- Un mécanisme de retry sera mis en place de façon recursive.

## Exercice 2

- Faire une fonction permettant de supprimer tous les espaces supperflus d'une string
- Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,
- Récupérer le domaine en fonction d'un url

In [23]:
import re
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def remove_extra_spaces(input_string):
    return re.sub(r'\s+', ' ', input_string).strip()

def html_to_text(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    text_content = soup.get_text(separator=' ')
    clean_text = re.sub(r'\s+', ' ', text_content).strip()
    return clean_text

def get_domain_from_url(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

In [24]:
original_string = "   Ceci    est    un    exemple    avec   des   espaces    superflus.   "
processed_string = remove_extra_spaces(original_string)
print("String après suppression des espaces superflus:", processed_string)

String après suppression des espaces superflus: Ceci est un exemple avec des espaces superflus.


In [25]:
html_string = "<p>Ceci est <b>un exemple</b> avec <a href='#'>du HTML</a>.</p>"
text_result = html_to_text(html_string)
print("Texte intelligible à partir du HTML:", text_result)

Texte intelligible à partir du HTML: Ceci est un exemple avec du HTML .


In [26]:
url_example = "https://www.example.com/path/to/page"
domain_result = get_domain_from_url(url_example)
print("Domaine extrait de l'URL:", domain_result)

Domaine extrait de l'URL: www.example.com


# Exploitation du HTML  

Ici, il faut récupérer le code HTML d'un site web à partir d'une requête. Lorsque vous avez récupéré le texte d'un site il faut le parser. Pour cela, on utilise BeautifulSoup qui permet de transformer la structure HTML en objet Python. Cela permet de récupérer efficacement les données qui nous intéresse.

Pour les webmasters, le blocage le plus souvent mis en place et un blocage sur le User-Agent. Le User-Agent est un paramètre intégré dans la requête HTTP réalisé par le Navigateur pour envoyer au front des informations basiques :

- la version du Navigateur,
- la version de l'OS
- Le type de gestionnaire graphique (Gecko)
- le type de device utilisé

Exemple de User Agent :  

`Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0`

Commençons à utiliser `BeautifulSoup`, pour l'installer : 

In [27]:
!pip install bs4
!pip install  lxml

In [28]:
import requests
from bs4 import BeautifulSoup

Pour transformer une requête (requests) en objet BeautifulSoup :

In [29]:
response = requests.get(url)
soup = BeautifulSoup(response.text)

Il se peut qu'un message d'erreur arrive à ce point là si vous n'avez pas la librarie `lxml` installée, pour se faire vous avez juste à lancer la commande suivante : 

In [30]:
!pip install lxml

Pour trouver tous les liens d'une page on récupère la balise `a` qui permet de gérer les liens en HTML  :

In [31]:
soup.find_all("a")[0:10]

[<a href="/#content">Aller au contenu</a>,
 <a href="/#menu">Aller au menu</a>,
 <a href="/plan-du-site/">Plan du site</a>,
 <a href="/nous-rencontrer-1">Bien choisir son école, c'est aussi la rencontrer  ! Rendez-vous à la prochaine journée portes ouvertes le 9 décembre.</a>,
 <a href="/"><img alt="ESIEE PARIS" class="a42-ac-replace-img" src="/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg"/></a>,
 <a href="/brochures-1">Brochures</a>,
 <a href="/informations/etudiantes-et-etudiants">Espace élèves</a>,
 <a href="/" hreflang="fr-FR" title="Français">
 <span>Fr</span>
 </a>,
 <a href="/en/" hreflang="en-US" title="English">
 <span>En</span>
 </a>,
 <a href="/candidater-1">Candidater</a>]

On peut préciser la classe HTML voulue  pour l'ensemble des `a`:

```python
soup.find_all(class_="<CLASS_NAME>")[0:10]
```

Ici par exemple: 

In [40]:
soup.find_all(class_="header")[0:5]

[<header class="header">
 <div class="skip-links" role="region">
 <a href="/#content">Aller au contenu</a>
 <a href="/#menu">Aller au menu</a>
 <a href="/plan-du-site/">Plan du site</a>
 </div>
 <div class="alerte" id="esieealert-CC01" role="alert">
 <div class="container">
 <div class="d-flex">
 <div class="d-flex relative">
 <svg class="iconpack fas fa-bell" data-iconfig="fa6:solid,bell" fill="currentColor" role="img" viewbox="0 0 448 512"><path d="M224 0c-17.7 0-32 14.3-32 32V51.2C119 66 64 130.6 64 208v18.8c0 47-17.3 92.4-48.5 127.6l-7.4 8.3c-8.4 9.4-10.4 22.9-5.3 34.4S19.4 416 32 416H416c12.6 0 24-7.4 29.2-18.9s3.1-25-5.3-34.4l-7.4-8.3C401.3 319.2 384 273.9 384 226.8V208c0-77.4-55-142-128-156.8V32c0-17.7-14.3-32-32-32zm45.3 493.3c12-12 18.7-28.3 18.7-45.3H224 160c0 17 6.7 33.3 18.7 45.3s28.3 18.7 45.3 18.7s33.3-6.7 45.3-18.7z"></path></svg>
 <div>
 <p><a href="/nous-rencontrer-1">Bien choisir son école, c'est aussi la rencontrer  ! Rendez-vous à la prochaine journée portes ouverte

Pour récupérer le text sans les balises HTML :

In [41]:
soup.text[0:1000]

"\n\n\n\nESIEE Paris, l'école de l'innovation technologique | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller au contenu\nAller au menu\nPlan du site\n\n\n\n\n\n\n\nBien choisir son école, c'est aussi la rencontrer  ! Rendez-vous à la prochaine journée portes ouvertes le 9 décembre.\n\n\n\n\n\nMasquer l'alerte\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrochuresEspace élèves\n\n\n\nFr\n\n\n\n\nEn\n\n\n\n\n\n\n\n\n\nAfficherMasquer la recherche\r\n\t\t\n\n\n\nSaisissez votre recherche\xa0:\n\nLancer la recherche\n\n\n\nCandidater\n\nAfficherMasquer le menu\n\n\n\n\n\nRetour au menu principalAfficherMasquer le sous menu\xa0: L'écolePourquoi choisir ESIEE Paris ?AfficherMasquer le sous menu\xa0: Gouvernance et conseilsGouvernance et conseilsConseil scientifiqueAfficherMasquer le sous menu\xa0: Départements d'enseignements et de rechercheInformatique et télécommunicationsIngénierie des systèmes cyberphysiquesIngénierie industrielleSanté, énergie et environnement durableManagement, sci

## Exercice
### Exercice 3

Améliorer la classe développé précédemment.

- Ajouter une méthode pour récupérer l'objet soup d'un url
- Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
- Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal.

Parsing d'un sitemaps pour récupérer une listes de liens avec les informations disponibles. -> Stocker dans un dictionnaire et dans un fichier JSON local.

In [43]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import random
import json

class CustomRequest:
    def __init__(self, user_agents=None, timeout=5, max_retries=3):
        self.user_agents = user_agents or ["UserAgent1/1.0", "UserAgent2/2.0", "UserAgent3/3.0"]
        self.timeout = timeout
        self.max_retries = max_retries
        self.session = self._create_session()

    def _create_session(self):
        session = requests.Session()
        retries = Retry(total=self.max_retries, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retries)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        return session

    def _make_request(self, url):
        headers = {'User-Agent': random.choice(self.user_agents)}
        try:
            response = self.session.get(url, headers=headers, timeout=self.timeout)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"Error making request: {e}")
            return None

    def get_soup_from_url(self, url):
        response = self._make_request(url)
        if response:
            return BeautifulSoup(response.content, 'html.parser')
        return None

    def parse_html_page(self, url):
        soup = self.get_soup_from_url(url)
        if soup:
            title = soup.title.text.strip() if soup.title else None
            h1_tags = [h1.text.strip() for h1 in soup.find_all('h1')]
            image_links = [img['src'] for img in soup.find_all('img', src=True)]
            outgoing_links = [a['href'] for a in soup.find_all('a', href=True) if 'http' in a['href']]
            main_text = soup.get_text(separator=' ').strip()

            parsed_data = {
                'title': title,
                'h1_tags': h1_tags,
                'image_links': image_links,
                'outgoing_links': outgoing_links,
                'main_text': main_text
            }

            return parsed_data
        return None

    def parse_sitemap(self, sitemap_url):
        soup = self.get_soup_from_url(sitemap_url)
        if soup:
            urls = [loc.text for loc in soup.find_all('loc')]
            data = {}
            for url in urls:
                parsed_data = self.parse_html_page(url)
                data[url] = parsed_data

            # Save the data to a local JSON file
            with open('sitemap_data.json', 'w') as json_file:
                json.dump(data, json_file, indent=2)

In [46]:
if __name__ == "__main__":
    custom_request = CustomRequest()
    parsed_data = custom_request.parse_html_page(url)
    print(parsed_data)

    custom_request.parse_sitemap(url)

{'title': "ESIEE Paris, l'école de l'innovation technologique | ESIEE Paris", 'h1_tags': [''], 'image_links': ['/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/2/4/csm_ESIEE-Home-Main-Picture_ff09bd6b6e.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/1/e/csm_eleves-groupe-galerie-1344x840_0f06698b28.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/6/2/csm_forum-descartes-rue-2019-1344x840_eb74cce508.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/c/8/csm_illustration-concours-jeunes-talents-orange-2023-1344x840_b06c9809ff.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/c/1/csm_actu-cl

# Exploitation des appels d'API



Losque le front du site récupère des données sur une API géré par le back, un appel d'API est réalisé. Cet appel est recensé dans les appels réseaux. Il est alors possible de re-jouer cet appel pour récupérer à nouveau les données. Il est très facile de récupérer ces appels dans l'onglet Network de la console développeur de Chrome ou FireFox. La console vous permet de copier le code CURL pour effectuée et vous pouvez ensuite la transformer en code Python depuis le site https://curl.trillworks.com/.

Souvent les APIs sont bloquées avec certain paramètres. L'API verifie que dans les headers de la requêtes HTTP ces paramètres sont présents : * un token généré à la volée avec des protocole OAuth2 (ou moins développés). * un referer provenant du site web (la source de la requête), très facile à falsifier.



## Exercice 
### Exercice 4

- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête
sur Qwant. 

Tips : 

- Aller sur https://www.qwant.com/
- Ouvrir les outils de développements de Chrome ou Firefox
- Onglet Network
- Fouiller dans les requêtes

In [52]:
def search_qwant(query):
    # URL de recherche Qwant
    url = f"https://www.qwant.com/?q={query}"

    # En-têtes HTTP pour simuler une requête depuis un navigateur
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Faire la requête HTTP
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupérer les premiers résultats (titres et liens)
    results = []
    for result in soup.select('.result__web'):
        title = result.select_one('.result__title').text.strip()
        link = result.select_one('.result__url').get('href')
        results.append({'title': title, 'link': link})

    return results

In [54]:
urlQwant = "https://www.qwant.com"
custom_request = CustomRequest()
user_agent = custom_request.rotate_user_agent()
headers = {'User-Agent': user_agent}
   
response = requests.get(urlQwant, headers=headers)
if response.status_code == 200:
    print("La requête a réussi.")
    soup = BeautifulSoup(response.text, 'html.parser')
    result_titles = [result.text.strip() for result in soup.find_all('a')]

    print(result_titles)

AttributeError: 'CustomRequest' object has no attribute 'rotate_user_agent'

# Exercice Final  

Exercice Final
Utilisez tout ce que vous avez appris pour récupérer des articles de News avec une catégorie. Il est souvent intéressant de partir des flux RSS pour commencer :

Les données doivent comprendre :
- Le texte important propre
- L'url
- Le domaine
- la catégorie
- Le titre de l'article
- Le titre de la page
- (Facultatif) : les images

Tips : 

- Taper le nom de votre média favoris + RSS (par exemple : https://www.lemonde.fr/rss/)
- Aller dans le DOM de la page 
- Trouver les catégories et les liens vers les articles